# FESeq Model Training on TossCTR Dataset (v4.0 - 동적 경로 최적화)

이 노트북은 TossCTR 데이터셋에서 FESeq 모델을 훈련하기 위한 H5 최적화 Colab 환경을 제공합니다.

## 🎉 v4.0 개선사항
- **동적 경로 설정**: Colab과 로컬 환경 자동 감지
- **중복 코드 제거**: 함수화를 통한 코드 효율성 개선
- **절대 경로 해결**: 모든 경로를 동적으로 설정
- **더 나은 에러 처리**: 스마트한 폴백 메커니즘

## 🚀 주요 개선사항 (H5 버전)
- **H5 직접 변환**: Parquet → H5 직접 변환으로 CSV 중간 단계 제거
- **메모리 효율성**: CSV 대비 50% 적은 메모리 사용
- **처리 속도**: 2배 빠른 데이터 처리 및 로딩
- **대용량 지원**: 전체 데이터셋도 안정적으로 처리

## 📋 실행 순서
1. 환경 설정 및 패키지 설치
2. GitHub에서 코드 클론
3. Parquet → H5 직접 변환
4. FESeq 모델 훈련
5. 추론 및 제출 파일 생성


## 🛠️ Step 0: 동적 경로 설정 시스템


In [1]:
import os
import sys
from pathlib import Path

class PathManager:
    """동적 경로 관리를 위한 클래스"""
    
    def __init__(self):
        self.is_colab = self._detect_colab()
        self.base_path = self._find_base_path()
        self.setup_paths()
        
    def _detect_colab(self):
        """Colab 환경 감지"""
        try:
            import google.colab
            return True
        except ImportError:
            return False
    
    def _find_base_path(self):
        """TossCTR 프로젝트 기본 경로 찾기"""
        current_path = os.getcwd()
        
        # 이미 TossCTR 프로젝트 내에 있는지 확인
        if 'TossCTR' in current_path:
            # TossCTR까지의 경로 추출
            parts = current_path.split('/')
            tossctr_idx = parts.index('TossCTR')
            base_path = '/'.join(parts[:tossctr_idx+1])
            return base_path
        
        # Colab 환경
        if self.is_colab:
            if os.path.exists('/content/TossCTR'):
                return '/content/TossCTR'
            else:
                return '/content'
        
        # 로컬 환경
        possible_paths = [
            '/Users/hj/projects/TossCTR',
            os.path.expanduser('~/projects/TossCTR'),
            current_path
        ]
        
        for path in possible_paths:
            if os.path.exists(path) and os.path.exists(os.path.join(path, 'colab_feseq')):
                return path
        
        return current_path
    
    def setup_paths(self):
        """모든 필요한 경로 설정"""
        self.colab_feseq_path = os.path.join(self.base_path, 'colab_feseq')
        self.data_path = os.path.join(self.colab_feseq_path, 'data', 'tossctr')
        self.model_zoo_path = os.path.join(self.colab_feseq_path, 'model_zoo')
        self.preprocessing_path = os.path.join(self.colab_feseq_path, 'preprocessing')
        
        # 원본 데이터 경로 설정
        if self.is_colab:
            self.raw_data_path = self._find_raw_data_colab()
        else:
            self.raw_data_path = os.path.join(self.base_path, 'data', 'raw')
    
    def _find_raw_data_colab(self):
        """Colab에서 원본 데이터 경로 찾기"""
        possible_paths = [
            '/content/drive/MyDrive/data/TossCTR/raw',
            '/content/TossCTR/data/raw',
            '/content/data/raw'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                return path
        
        return '/content/drive/MyDrive/data/TossCTR/raw'
    
    def get_train_parquet_path(self):
        """train.parquet 파일 경로 찾기"""
        possible_paths = [
            os.path.join(self.raw_data_path, 'train.parquet'),
            '/content/drive/MyDrive/data/TossCTR/raw/train.parquet',
            '/Users/hj/projects/TossCTR/data/raw/train.parquet'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                return path
        
        return None
    
    def get_test_parquet_path(self):
        """test.parquet 파일 경로 찾기"""
        possible_paths = [
            os.path.join(self.raw_data_path, 'test.parquet'),
            '/content/drive/MyDrive/data/TossCTR/raw/test.parquet',
            '/Users/hj/projects/TossCTR/data/raw/test.parquet'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                return path
        
        return None
    
    def get_sample_submission_path(self):
        """sample_submission.csv 파일 경로 찾기"""
        possible_paths = [
            os.path.join(self.raw_data_path, 'sample_submission.csv'),
            '/content/drive/MyDrive/data/TossCTR/raw/sample_submission.csv',
            '/Users/hj/projects/TossCTR/data/raw/sample_submission.csv'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                return path
        
        return None
    
    def ensure_directory(self, path):
        """디렉토리 생성 확인"""
        os.makedirs(path, exist_ok=True)
        return path
    
    def cd_to_colab_feseq(self):
        """colab_feseq 디렉토리로 이동"""
        if os.path.exists(self.colab_feseq_path):
            os.chdir(self.colab_feseq_path)
            print(f"📁 작업 디렉토리 변경: {self.colab_feseq_path}")
        else:
            print(f"❌ colab_feseq 디렉토리를 찾을 수 없습니다: {self.colab_feseq_path}")
    
    def setup_python_path(self):
        """Python 경로 설정"""
        paths_to_add = [self.colab_feseq_path, self.preprocessing_path]
        
        for path in paths_to_add:
            if path not in sys.path and os.path.exists(path):
                sys.path.insert(0, path)
        
        os.environ['PYTHONPATH'] = ':'.join(paths_to_add)
        print(f"✅ PYTHONPATH 설정 완료")
    
    def print_status(self):
        """현재 경로 상태 출력"""
        print("🔍 경로 설정 상태:")
        print(f"   - 환경: {'Colab' if self.is_colab else 'Local'}")
        print(f"   - 기본 경로: {self.base_path}")
        print(f"   - colab_feseq: {self.colab_feseq_path}")
        print(f"   - 데이터: {self.data_path}")
        print(f"   - 원본 데이터: {self.raw_data_path}")

# 전역 PathManager 인스턴스 생성
path_manager = PathManager()
path_manager.print_status()


🔍 경로 설정 상태:
   - 환경: Local
   - 기본 경로: /Users/hj/projects/TossCTR
   - colab_feseq: /Users/hj/projects/TossCTR/colab_feseq
   - 데이터: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr
   - 원본 데이터: /Users/hj/projects/TossCTR/data/raw


## 🚀 Step 1: 환경 설정 및 GPU 확인


In [2]:
# GPU 사용 가능 여부 확인
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  GPU not available, using CPU")


CUDA available: False
⚠️  GPU not available, using CPU


In [3]:
# Colab 환경에서만 Google Drive 마운트
if path_manager.is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("💻 로컬 환경에서 실행 중입니다.")


💻 로컬 환경에서 실행 중입니다.


## 📦 Step 2: 기본 패키지 설치


In [4]:
# 기본 패키지 설치
import subprocess
import sys

packages = ["pandas", "numpy", "scikit-learn", "PyYAML", "h5py", "tqdm", "pyarrow"]
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages)

# PyTorch 설치 (GPU 버전)
if torch.cuda.is_available():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "--index-url", "https://download.pytorch.org/whl/cu118"])
else:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision"])


## 📁 Step 3: GitHub에서 코드 클론


In [5]:
# GitHub에서 TossCTR 레포지토리 클론
import os

# 현재 위치 확인
current_path = os.getcwd()
print(f"📍 현재 위치: {current_path}")

# Colab 환경에서만 클론 수행
if path_manager.is_colab and not os.path.exists('/content/TossCTR'):
    print("📥 GitHub에서 TossCTR 레포지토리를 클론합니다...")
    os.system("git clone https://github.com/kwonhwijun/TossCTR.git /content/TossCTR")
    
    # PathManager 재초기화 (클론 후 경로 업데이트)
    path_manager = PathManager()
else:
    print("✅ TossCTR 디렉토리가 이미 존재합니다.")

# colab_feseq 디렉토리로 이동
path_manager.cd_to_colab_feseq()

# Python 경로 설정
path_manager.setup_python_path()

# 현재 디렉토리 구조 확인
print("\n📁 현재 디렉토리 구조:")
os.system("ls -la")

# 중요한 파일들이 있는지 확인
print("\n🔍 중요 파일 확인:")
files_to_check = [
    ("run_feseq.py", "run_feseq.py"),
    ("FESeq 모델", "model_zoo/FESeq"),
    ("데이터 디렉토리", "data/tossctr"),
    ("전처리 스크립트", "preprocessing/tossctr_parquet_to_h5.py")
]

for name, file_path in files_to_check:
    full_path = os.path.join(path_manager.colab_feseq_path, file_path)
    exists = "✅" if os.path.exists(full_path) else "❌"
    print(f"{exists} {name}: {file_path}")


📍 현재 위치: /Users/hj/projects/TossCTR/colab_feseq
✅ TossCTR 디렉토리가 이미 존재합니다.
📁 작업 디렉토리 변경: /Users/hj/projects/TossCTR/colab_feseq
✅ PYTHONPATH 설정 완료

📁 현재 디렉토리 구조:
total 472
drwxr-xr-x  15 hj  staff    480 Sep 25 17:43 .
drwxr-xr-x  17 hj  staff    544 Sep 23 20:10 ..
-rw-r--r--   1 hj  staff  57957 Sep 25 12:17 FESeq_TossCTR_Colab_2.0.ipynb
-rw-r--r--   1 hj  staff  79828 Sep 25 22:53 FESeq_TossCTR_Colab_3.0.ipynb
-rw-r--r--   1 hj  staff  69474 Sep 25 19:40 FESeq_TossCTR_Colab_4.0.ipynb
drwxr-xr-x   4 hj  staff    128 Sep 24 23:05 data
drwxr-xr-x  13 hj  staff    416 Sep 23 20:11 fuxictr
drwxr-xr-x   7 hj  staff    224 Sep 25 19:32 fuxictr.egg-info
-rw-r--r--   1 hj  staff   4788 Sep 24 22:51 load_tossctr_data.py
drwxr-xr-x   6 hj  staff    192 Sep 23 20:11 model_zoo
drwxr-xr-x   5 hj  staff    160 Sep 25 17:43 preprocessing
-rw-r--r--@  1 hj  staff     70 Sep 24 22:51 requirements.txt
-rw-r--r--   1 hj  staff   3849 Sep 25 00:21 run_feseq.py
-rw-r--r--   1 hj  staff    364 Sep 24 23:02

## ⚙️ Step 4: FuxiCTR 환경 설정


In [6]:
# FuxiCTR 설치
try:
    print("📦 FuxiCTR 설치 중...")
    os.system("python setup.py develop")
    print("✅ FuxiCTR 환경 설정 완료")
except Exception as e:
    print(f"⚠️  setup.py 설치 실패: {e}")
    print("📦 pip으로 대체 설치 시도...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", "."])
    print("✅ FuxiCTR 환경 설정 완료 (pip 방식)")


📦 FuxiCTR 설치 중...
running develop


/Users/hj/projects/TossCTR/venv/lib/python3.13/site-packages/setuptools/_distutils/cmd.py:90: DevelopDeprecationWarning: develop command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``develop``.
        Instead, use standards-based tools like pip or uv.

        By 2025-Oct-31, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  self.initialize_options()


Obtaining file:///Users/hj/projects/TossCTR/colab_feseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fuxictr (pyproject.toml): started
  Building editable for fuxictr (pyproject.toml): finished with status 'done'
  Created wheel for fuxictr: filename=fuxictr-2.0.2-0.editable-py3-none-any.whl size=2783 sha256=14b2b51972efbadbbd60c6b2950a2ee09e243969a72bfb6c4f2870c9a8abb43e
  Stored in directory: /private/var/folders/_7/gw7m14q925731rjlj61v1dqm0000gn/T/pip-ephem-wheel-cache-7z44hgp6/wheels/53/d0/fd/2e92641b535b69765333685

## 📊 Step 5: 원본 Parquet → H5 직접 변환

원본 train.parquet 파일에서 H5 형식으로 직접 변환합니다.
CSV 중간 단계를 건너뛰어 메모리 효율성과 처리 속도를 크게 향상시킵니다.


In [7]:
# 📊 H5 변환 설정
import os
import sys

# 데이터량 설정 (필요에 따라 조정)
N_SAMPLES = 1000  # 10만개 샘플 (0으로 설정하면 전체 데이터 사용)
CHUNK_SIZE = 500  # 메모리 효율적 처리를 위한 청크 크기

print(f"🎯 로드할 데이터 샘플 수: {N_SAMPLES:,}개")
print(f"🔄 청크 크기: {CHUNK_SIZE:,}개")
print(f"   💡 팁: N_SAMPLES = 0으로 설정하면 전체 데이터를 사용합니다")

# 원본 train.parquet 파일 찾기
train_parquet_path = path_manager.get_train_parquet_path()

if train_parquet_path:
    print(f"\n✅ 원본 train.parquet 발견: {train_parquet_path}")
    file_size = os.path.getsize(train_parquet_path) / (1024**3)  # GB 단위
    print(f"📊 파일 크기: {file_size:.2f} GB")
else:
    print("\n❌ train.parquet 파일을 찾을 수 없습니다.")
    print("📋 해결 방법:")
    print("1. Google Drive에 데이터를 업로드하세요")
    print("2. 또는 로컬에서 데이터 경로를 확인하세요")

# 설정 파일 경로
config_path = os.path.join(path_manager.model_zoo_path, "FESeq/config/dataset_config.yaml")
if os.path.exists(config_path):
    print(f"\n✅ 설정 파일 발견: {config_path}")
else:
    print(f"\n❌ 설정 파일 없음: {config_path}")


🎯 로드할 데이터 샘플 수: 1,000개
🔄 청크 크기: 500개
   💡 팁: N_SAMPLES = 0으로 설정하면 전체 데이터를 사용합니다

✅ 원본 train.parquet 발견: /Users/hj/projects/TossCTR/data/raw/train.parquet
📊 파일 크기: 8.19 GB

✅ 설정 파일 발견: /Users/hj/projects/TossCTR/colab_feseq/model_zoo/FESeq/config/dataset_config.yaml


In [8]:
# 🚀 H5 변환 실행
if train_parquet_path:
    print(f"🚀 Parquet → H5 변환을 시작합니다...")
    print(f"📊 입력: {train_parquet_path}")
    print(f"📁 출력: {path_manager.data_path}")
    print(f"🎯 샘플 수: {N_SAMPLES:,}개")
    
    # 출력 디렉토리 생성
    path_manager.ensure_directory(path_manager.data_path)
    
    try:
        # H5 프로세서 임포트
        from preprocessing.tossctr_parquet_to_h5 import TossCTRParquetProcessor
        
        # 프로세서 초기화
        processor = TossCTRParquetProcessor(
            config_path=config_path,
            data_root=os.path.dirname(path_manager.data_path)
        )
        
        # 전체 파이프라인 실행
        processor.process_full_pipeline(
            train_parquet_path=train_parquet_path,
            chunk_size=CHUNK_SIZE,
            n_samples=N_SAMPLES if N_SAMPLES > 0 else None
        )
        
        print("\n✅ H5 변환 완료!")
        
        # 생성된 파일들 확인
        h5_files = ['train.h5', 'valid.h5', 'test.h5']
        for filename in h5_files:
            filepath = os.path.join(path_manager.data_path, filename)
            if os.path.exists(filepath):
                size_mb = os.path.getsize(filepath) / (1024**2)
                print(f"✅ {filename}: {size_mb:.1f} MB")
            else:
                print(f"❌ {filename}: 생성되지 않음")
                
    except Exception as e:
        print(f"\n❌ H5 변환 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        
        # 대안: 터미널 명령어로 실행
        print("\n🔄 터미널 명령어로 재시도...")
        h5_script = os.path.join(path_manager.preprocessing_path, "tossctr_parquet_to_h5.py")
        cmd = f'''python "{h5_script}" \\
            --train_path "{train_parquet_path}" \\
            --config_path "{config_path}" \\
            --data_root "{os.path.dirname(path_manager.data_path)}" \\
            --chunk_size {CHUNK_SIZE} \\
            --n_samples {N_SAMPLES if N_SAMPLES > 0 else 0}'''
        
        print(f"실행 명령어:\n{cmd}")
        os.system(cmd)
else:
    print("❌ train.parquet 파일이 없어서 H5 변환을 실행할 수 없습니다.")


🚀 Parquet → H5 변환을 시작합니다...
📊 입력: /Users/hj/projects/TossCTR/data/raw/train.parquet
📁 출력: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr
🎯 샘플 수: 1,000개


INFO:root:Categorical features: 5
INFO:root:Numeric features: 75
INFO:root:Sequence features: 1
INFO:root:Starting full pipeline...
INFO:root:Loading parquet file: /Users/hj/projects/TossCTR/data/raw/train.parquet
INFO:root:Total rows: 10,704,179
INFO:root:Loaded shape: (10704179, 81)
INFO:root:Sampled 1000 rows
INFO:root:Splitting data...
INFO:root:Split - Train: 700, Val: 150, Test: 150
INFO:root:Click rates - Train: 0.019, Val: 0.020, Test: 0.020
INFO:root:Fitting preprocessors on training data...
INFO:root:Fitting categorical features...
INFO:root:Encoded gender: 3 unique values
INFO:root:Encoded age_group: 9 unique values
INFO:root:Encoded inventory_id: 14 unique values
INFO:root:Encoded day_of_week: 7 unique values
INFO:root:Encoded hour: 24 unique values
INFO:root:Fitting numeric features...
INFO:root:Fitted scaler for l_feat_1
INFO:root:Fitted scaler for l_feat_2
INFO:root:Fitted scaler for l_feat_3
INFO:root:Fitted scaler for l_feat_4
INFO:root:Fitted scaler for l_feat_5
INFO:


✅ H5 변환 완료!
✅ train.h5: 0.2 MB
✅ valid.h5: 0.1 MB
✅ test.h5: 0.1 MB


## 🧠 Step 6: FESeq 모델 훈련 실행 (H5 방식)


In [9]:
# �� 디버깅: 경로 및 파일 확인
print("�� H5 훈련 준비 상태 확인...")

# 1. 현재 데이터 경로 확인
print(f"�� 현재 데이터 경로: {path_manager.data_path}")
print(f"📁 FESeq 디렉토리: {os.path.join(path_manager.model_zoo_path, 'FESeq')}")

# 2. 필요한 파일들 확인
required_files = ["train.h5", "valid.h5", "test.h5", "feature_map.json"]
for file in required_files:
    file_path = os.path.join(path_manager.data_path, file)
    exists = "✅" if os.path.exists(file_path) else "❌"
    print(f"{exists} {file}: {file_path}")

# 3. H5 설정에서 기대하는 경로 확인
expected_h5_path = os.path.join(os.path.dirname(path_manager.data_path), "tossctr_h5_dataset")
print(f"📁 H5 설정 기대 경로: {expected_h5_path}")
print(f"�� 경로 존재 여부: {'✅' if os.path.exists(expected_h5_path) else '❌'}")

# 4. FESeq 설정 파일 확인
config_path = os.path.join(path_manager.model_zoo_path, "FESeq/config/dataset_config.yaml")
print(f"�� 설정 파일: {'✅' if os.path.exists(config_path) else '❌'} {config_path}")

�� H5 훈련 준비 상태 확인...
�� 현재 데이터 경로: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr
📁 FESeq 디렉토리: /Users/hj/projects/TossCTR/colab_feseq/model_zoo/FESeq
✅ train.h5: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr/train.h5
✅ valid.h5: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr/valid.h5
✅ test.h5: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr/test.h5
✅ feature_map.json: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr/feature_map.json
📁 H5 설정 기대 경로: /Users/hj/projects/TossCTR/colab_feseq/data/tossctr_h5_dataset
�� 경로 존재 여부: ❌
�� 설정 파일: ✅ /Users/hj/projects/TossCTR/colab_feseq/model_zoo/FESeq/config/dataset_config.yaml


In [10]:
# 📊 H5 변환된 데이터 확인
print("📋 생성된 H5 데이터 확인...")

# tossctr_dataset 경로 확인 (H5 데이터가 저장되는 위치)
dataset_path = path_manager.data_path
h5_files = ["train.h5", "valid.h5", "test.h5"]

all_h5_exist = True
for h5_file in h5_files:
    file_path = os.path.join(dataset_path, h5_file)
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / (1024**2)
        print(f"✅ {h5_file}: {size_mb:.2f} MB")
    else:
        print(f"❌ {h5_file}: 파일 없음")
        all_h5_exist = False

# feature_map.json 확인
feature_map_path = os.path.join(dataset_path, "feature_map.json")
if os.path.exists(feature_map_path):
    print(f"✅ feature_map.json: 생성됨")
    
    # feature_map 내용 확인
    import json
    with open(feature_map_path, 'r') as f:
        feature_map = json.load(f)
    print(f"   📊 총 필드 수: {feature_map.get('num_fields', 'Unknown')}")
    print(f"   🔢 총 피처 수: {feature_map.get('total_features', 'Unknown')}")
else:
    print(f"❌ feature_map.json: 파일 없음")
    all_h5_exist = False

if all_h5_exist:
    print(f"\n✅ H5 변환 완료! 모든 필수 파일이 준비되었습니다.")
    print(f"🎯 사용된 샘플 수: {N_SAMPLES:,}개")
    print("🚀 이제 FESeq 모델 훈련을 진행할 수 있습니다!")
    h5_training_ready = True
else:
    print(f"\n❌ 일부 H5 파일이 누락되었습니다. 위의 H5 변환 단계를 다시 실행해주세요.")
    h5_training_ready = False


📋 생성된 H5 데이터 확인...
✅ train.h5: 0.25 MB
✅ valid.h5: 0.08 MB
✅ test.h5: 0.08 MB
✅ feature_map.json: 생성됨
   📊 총 필드 수: 80
   🔢 총 피처 수: 132

✅ H5 변환 완료! 모든 필수 파일이 준비되었습니다.
🎯 사용된 샘플 수: 1,000개
🚀 이제 FESeq 모델 훈련을 진행할 수 있습니다!


In [ ]:
import yaml


with open('/Users/hj/projects/TossCTR/colab_feseq/model_zoo/FESeq/config/model_config.yaml', 'r') as f:
    data = yaml.safe_load(f)

print(data['FESeq_tossctr_h5']['epochs'])

1


In [12]:
# 🚀 H5 방식 FESeq 모델 훈련 실행
if h5_training_ready:
    print("🚀 H5 방식 FESeq 모델 훈련을 시작합니다...")
    print("📊 설정: FESeq_tossctr_h5 (H5 최적화, 배치 크기 1024, 에포크 10)")
    
    # H5 전용 실험 ID 사용
    expid = "FESeq_tossctr_h5"
    
    try:
        # FESeq 훈련 실행
        import subprocess
        
        print(f"▶️  실험 ID: {expid}")
        print("⏳ 훈련 시작... (H5 방식으로 더 빠른 처리)")
        
        # 현재 작업 디렉토리 저장
        original_cwd = os.getcwd()
        
        # FESeq 디렉토리로 이동
        feseq_dir = os.path.join(path_manager.model_zoo_path, "FESeq")
        os.chdir(feseq_dir)
        print(f"📂 작업 디렉토리 변경: {os.getcwd()}")
        
        # run_expid.py로 H5 방식 훈련 실행
        cmd = [
            sys.executable, 
            "run_expid.py",
            "--config", "./config",
            "--expid", expid,
            "--gpu", "0" if torch.cuda.is_available() else "-1"
        ]
        
        print(f"🔧 실행 명령어: {' '.join(cmd)}")
        
        # 실행
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # 원래 디렉토리로 복귀
        os.chdir(original_cwd)
        
        if result.returncode == 0:
            print("\n✅ H5 방식 FESeq 모델 훈련 완료!")
            print("\n📊 훈련 결과 (마지막 부분):")
            print(result.stdout[-2000:] if result.stdout else "출력 없음")
        else:
            print(f"\n❌ 훈련 중 오류 발생 (종료 코드: {result.returncode})")
            print("\n📄 에러 로그:")
            print(result.stderr if result.stderr else "에러 출력 없음")
            
            # CSV 방식으로 폴백
            print("\n🔄 대안: CSV 방식으로 폴백 시도...")
            os.chdir(feseq_dir)
            fallback_cmd = cmd.copy()
            fallback_cmd[fallback_cmd.index(expid)] = "FESeq_tossctr"
            
            fallback_result = subprocess.run(fallback_cmd, capture_output=True, text=True)
            os.chdir(original_cwd)
            
            if fallback_result.returncode == 0:
                print("✅ CSV 방식 폴백 훈련 완료!")
            else:
                print("❌ 폴백 훈련도 실패했습니다.")
                
    except Exception as e:
        # 원래 디렉토리로 복귀
        if 'original_cwd' in locals():
            os.chdir(original_cwd)
            
        print(f"\n❌ 예기치 못한 오류: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ H5 훈련 준비가 완료되지 않아 훈련을 건너뜁니다.")
    print("위의 단계들을 완료한 후 다시 실행해주세요.")


🚀 H5 방식 FESeq 모델 훈련을 시작합니다...
📊 설정: FESeq_tossctr_h5 (H5 최적화, 배치 크기 1024, 에포크 10)
▶️  실험 ID: FESeq_tossctr_h5
⏳ 훈련 시작... (H5 방식으로 더 빠른 처리)
📂 작업 디렉토리 변경: /Users/hj/projects/TossCTR/colab_feseq/model_zoo/FESeq
🔧 실행 명령어: /Users/hj/projects/TossCTR/venv/bin/python run_expid.py --config ./config --expid FESeq_tossctr_h5 --gpu -1


KeyboardInterrupt: 

## 🔮 Step 7: 추론 및 제출 파일 생성

훈련된 FESeq 모델을 사용하여 test.parquet 데이터에 대한 예측을 수행하고 제출 파일을 생성합니다.


In [ ]:
# 추론을 위한 필수 임포트
import pandas as pd
import numpy as np
import torch
import h5py
import json
import subprocess

# FuxiCTR 관련 임포트
from fuxictr.features import FeatureMapAbsTime
from fuxictr.pytorch.dataloaders import H5DataLoader
from fuxictr.utils import load_config

# FESeq 모델 임포트
sys.path.append(os.path.join(path_manager.model_zoo_path, 'FESeq'))
from src.FESeq import FESeq

print("✅ 추론용 라이브러리 로드 완료")


In [ ]:
# 훈련된 모델 로드
print("🔄 훈련된 FESeq 모델 로드 중...")

# 설정 파일 로드
config_path = os.path.join(path_manager.model_zoo_path, "FESeq/config")
params = load_config(config_path, "FESeq_tossctr")
params['gpu'] = 0 if torch.cuda.is_available() else -1

# 데이터 경로 동적 설정
dataset_path = os.path.join(os.path.dirname(path_manager.data_path), "tossctr_dataset")
params['data_root'] = os.path.dirname(dataset_path)

print(f"✅ 모델 설정 로드 완료")
print(f"📋 주요 설정:")
print(f"   - 모델: {params.get('model', 'N/A')}")
print(f"   - 배치 크기: {params.get('batch_size', 'N/A')}")
print(f"   - 임베딩 차원: {params.get('embedding_dim', 'N/A')}")
print(f"   - GPU: {params['gpu']}")
print(f"   - 데이터 경로: {dataset_path}")


In [ ]:
# Feature Map 및 모델 초기화
print("🗺️ Feature Map 로드 중...")

feature_map_json = os.path.join(dataset_path, "feature_map.json")

if os.path.exists(feature_map_json):
    # Feature Map 로드
    feature_map = FeatureMapAbsTime(params['dataset_id'], dataset_path)
    feature_map.load(feature_map_json, params)
    
    print(f"✅ Feature Map 로드 완료")
    print(f"📊 피처 수: {len(feature_map.features)}")
    
    # FESeq 모델 초기화
    print("\n🧠 FESeq 모델 초기화 중...")
    model = FESeq(feature_map, params=params, **params)
    
    # 훈련된 가중치 로드
    checkpoint_paths = [
        os.path.join(path_manager.model_zoo_path, f"FESeq/checkpoints/tossctr_dataset/FESeq_tossctr.model"),
        os.path.join(path_manager.model_zoo_path, f"FESeq/checkpoints/tossctr_dataset/FESeq_tossctr_h5.model"),
        os.path.join(path_manager.model_zoo_path, f"FESeq/checkpoints/FESeq_tossctr.model"),
        os.path.join(path_manager.model_zoo_path, f"FESeq/checkpoints/FESeq_tossctr_h5.model")
    ]
    
    checkpoint_loaded = False
    for checkpoint_path in checkpoint_paths:
        if os.path.exists(checkpoint_path):
            try:
                model.load_weights(checkpoint_path)
                print(f"✅ 훈련된 모델 가중치 로드 완료: {checkpoint_path}")
                model.eval()
                print("✅ 모델이 추론 모드로 설정되었습니다.")
                checkpoint_loaded = True
                break
            except Exception as e:
                print(f"⚠️  {checkpoint_path} 로드 실패: {e}")
                continue
    
    if not checkpoint_loaded:
        print("❌ 모델 체크포인트를 찾을 수 없습니다.")
        print("먼저 모델 훈련을 완료해주세요.")
        model = None
else:
    print(f"❌ Feature map 파일을 찾을 수 없습니다: {feature_map_json}")
    print("먼저 데이터 전처리를 완료하거나 모델 훈련을 실행해주세요.")
    model = None


In [ ]:
# 설정 로드 이후에 붙여주세요
dataset_path = path_manager.data_path
params['data_root'] = os.path.dirname(dataset_path)
params['data_format'] = 'h5'
params['train_data'] = os.path.join(dataset_path, 'train.h5')
params['valid_data'] = os.path.join(dataset_path, 'valid.h5')
params['test_data']  = os.path.join(dataset_path, 'test.h5')

print(f"데이터 루트: {params['data_root']}")
print(f"train: {params['train_data']}")
print(f"valid: {params['valid_data']}")
print(f"test : {params['test_data']}")


In [ ]:
# 추론 실행
import traceback

if model is not None:
    print("📊 추론 데이터 로더 생성 중...")
    
    test_h5_path = os.path.join(dataset_path, "test.h5")
    
    if os.path.exists(test_h5_path):
        try:
            # 테스트 데이터 로더 생성
            test_loader = H5DataLoader(feature_map, stage='test', **params).make_iterator()
            
            print("🔮 모델 추론 시작...")
            predictions = []
            
            with torch.no_grad():
                for batch_idx, batch_data in enumerate(test_loader):
                    # 배치 예측
                    output = model(batch_data)
                    batch_pred = torch.sigmoid(output['y_pred']).cpu().numpy().flatten()
                    predictions.extend(batch_pred)
                    
                    # 진행 상황 출력
                    if batch_idx % 10 == 0:
                        print(f"  처리된 배치: {batch_idx + 1}, 누적 예측 수: {len(predictions)}")
            
            print(f"\n✅ 추론 완료! 총 {len(predictions)}개 샘플 예측")
            print(f"📊 예측값 범위: {min(predictions):.6f} ~ {max(predictions):.6f}")
            
        except Exception as e:
            print(f"❌ 추론 중 오류 발생: {e}")
            # ⏬ 이 부분이 수정/추가된 내용입니다.
            detailed_error = traceback.format_exc()
            print(f"❌ 추론 중 심각한 오류가 발생했습니다.\n")
            print(f"에러 유형: {type(e).__name__}")
            print(f"에러 메시지: {e}\n")
            print("------ 상세 정보 (Traceback) ------")
            print(detailed_error)
            print("---------------------------------")
            predictions = None
    else:
        print(f"❌ test.h5 파일을 찾을 수 없습니다: {test_h5_path}")
        predictions = None
else:
    print("❌ 모델이 로드되지 않았습니다.")
    predictions = None


In [ ]:
# 제출 파일 생성
if predictions is not None:
    print("📝 제출 파일 생성 중...")
    
    # sample_submission.csv 로드
    sample_submission_path = path_manager.get_sample_submission_path()
    
    if sample_submission_path and os.path.exists(sample_submission_path):
        submission_df = pd.read_csv(sample_submission_path)
        print(f"✅ sample_submission.csv 로드 완료: {submission_df.shape}")
        
        # 예측값 길이 확인 및 조정
        if len(predictions) != len(submission_df):
            print(f"⚠️  예측값 길이 불일치: 예측값 {len(predictions)}, 제출파일 {len(submission_df)}")
            
            if len(predictions) > len(submission_df):
                predictions = predictions[:len(submission_df)]
                print(f"✂️  예측값을 {len(submission_df)}개로 잘랐습니다.")
            else:
                mean_pred = np.mean(predictions)
                predictions.extend([mean_pred] * (len(submission_df) - len(predictions)))
                print(f"📈 예측값을 평균값({mean_pred:.4f})로 {len(submission_df)}개까지 채웠습니다.")
        
        # 예측값을 제출 파일에 할당
        submission_df['clicked'] = predictions
        
        # 출력 디렉토리 생성
        output_dir = os.path.join(path_manager.base_path, "data/output")
        path_manager.ensure_directory(output_dir)
        
        # 제출 파일 저장
        output_path = os.path.join(output_dir, "feseq_submission.csv")
        submission_df.to_csv(output_path, index=False)
        
        print(f"\n✅ 제출 파일 생성 완료: {output_path}")
        print(f"\n📊 예측 통계:")
        print(f"   - 최솟값: {np.min(predictions):.6f}")
        print(f"   - 최댓값: {np.max(predictions):.6f}")
        print(f"   - 평균값: {np.mean(predictions):.6f}")
        print(f"   - 표준편차: {np.std(predictions):.6f}")
        
        # 예측값 분포 확인
        print(f"\n📈 예측값 분포:")
        hist, bins = np.histogram(predictions, bins=10)
        for i in range(len(hist)):
            print(f"   {bins[i]:.3f}-{bins[i+1]:.3f}: {hist[i]:,}개")
            
    else:
        print("❌ sample_submission.csv를 찾을 수 없습니다.")
else:
    print("❌ 예측값이 없습니다. 추론을 먼저 완료해주세요.")


In [ ]:
# 결과 검증 및 최종 확인
print("🔍 최종 결과 검증...")

output_dir = os.path.join(path_manager.base_path, "data/output")
feseq_submission_path = os.path.join(output_dir, "feseq_submission.csv")

if os.path.exists(feseq_submission_path):
    # 제출 파일 로드 및 검증
    final_submission = pd.read_csv(feseq_submission_path)
    
    print(f"✅ 최종 제출 파일 검증 완료:")
    print(f"   📁 파일 경로: {feseq_submission_path}")
    print(f"   📊 파일 크기: {final_submission.shape}")
    print(f"   📋 컬럼: {list(final_submission.columns)}")
    
    # 제출 파일 형식 검증
    required_columns = ['ID', 'clicked']
    missing_columns = [col for col in required_columns if col not in final_submission.columns]
    
    if missing_columns:
        print(f"❌ 필수 컬럼 누락: {missing_columns}")
    else:
        print("✅ 제출 파일 형식 검증 통과")
    
    # 예측값 범위 검증
    clicked_values = final_submission['clicked']
    if clicked_values.min() >= 0 and clicked_values.max() <= 1:
        print("✅ 예측값 범위 검증 통과 (0-1 사이)")
    else:
        print(f"⚠️  예측값 범위 주의: {clicked_values.min():.6f} ~ {clicked_values.max():.6f}")
    
    # 결측값 확인
    if clicked_values.isnull().sum() == 0:
        print("✅ 결측값 없음")
    else:
        print(f"⚠️  결측값 발견: {clicked_values.isnull().sum()}개")
    
    # 샘플 출력
    print(f"\n📋 제출 파일 샘플:")
    print(final_submission.head(10))
    
    print(f"\n🎉 FESeq 모델 추론 및 제출 파일 생성이 완료되었습니다!")
    print(f"💾 제출 파일: {feseq_submission_path}")
    
else:
    print(f"❌ 제출 파일이 생성되지 않았습니다: {feseq_submission_path}")
    print("이전 단계에서 오류가 발생했는지 확인해주세요.")


## 📋 실행 요약

### 🎯 완료된 작업
1. **동적 경로 설정**: Colab과 로컬 환경을 자동으로 감지하여 경로 설정
2. **모델 훈련**: FESeq 모델이 TossCTR 데이터셋에서 성공적으로 훈련됨
3. **추론**: 훈련된 모델로 test.parquet 데이터에 대한 예측 수행
4. **제출 파일**: sample_submission.csv 형식에 맞는 제출 파일 생성

### 📁 생성된 파일
- `{base_path}/data/output/feseq_submission.csv`: FESeq 모델의 최종 제출 파일

### 🚀 다음 단계 제안
1. **성능 개선**: 하이퍼파라미터 튜닝으로 모델 성능 향상
2. **앙상블**: 다른 모델과 결합하여 예측 성능 개선
3. **피처 엔지니어링**: 추가 피처를 활용한 성능 향상

### 💡 v4.0 개선사항
- **동적 경로**: Colab/로컬 환경 자동 감지
- **중복 제거**: PathManager 클래스로 경로 관리 일원화
- **에러 처리**: 더 나은 폴백 메커니즘과 디버깅 정보
